In [1]:
!pip install tensorflow==2.0
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 86.3MB 76kB/s 
     |████████████████████████████████| 3.8MB 44.7MB/s 
     |████████████████████████████████| 450kB 49.3MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=5cff621e7aebc9b61ced4af56bdbedac14499ae37c706d72b4854a0540aaaaff
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tenso

In [2]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

In [3]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [4]:
s = "I've always been trashed my whole life. When people don't see a reason to use me anymore, I get dumped, this shit happens since my first teenage years and idk why at all. I try to always be kind with everyone, I apologize for my mistakes, I try to help people around me but when I need no one fucking helps me. I'm tired of being alone, I'm tired of always being the third wheel, it just seems like everyone else is better and more important than me, i just want someone to actually care about me."

stokens = tokenizer.tokenize(s)

stokens = ["[CLS]"] + stokens + ["[SEP]"]

input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])

print(all_embs)

[[[ 0.41761225  0.18385038  0.16049394 ... -0.27042252  0.04498798
    0.30408818]
  [ 0.60110945  0.22380722 -0.22009708 ...  0.15909903  0.323846
    0.23461579]
  [ 0.65815234  0.1644309  -0.00857984 ... -0.11120957 -0.6318881
   -0.3440389 ]
  ...
  [ 0.3277028   0.28513843  0.206884   ...  0.14911777 -0.11892851
    0.19611841]
  [ 0.3743372   0.11093628  0.1335176  ...  0.20249088 -0.0333361
   -0.06254143]
  [ 0.2808527   0.06968231  0.2521266  ...  0.23377748 -0.11364549
   -0.10920064]]]


In [13]:
from tensorflow.keras.models import load_model
loadedModel = tf.keras.Model.load_weights('/content/bertmodelClustered.h5')

# load_model_path = "/content/bertmodelClustered.h5"

# if load_model_path.endswith('.h5'):
#         model = tf.keras.models.load_model(load_model_path)

predictions = loadedModel.predict(all_embs)
print(predictions)

TypeError: ignored